In [1]:
import sys
import numpy as np
import pandas as pd
from scipy import stats
pd.options.mode.chained_assignment = None 

In [ ]:
df_category = pd.read_excel('https://www.dropbox.com/s/jcdjj2znimqbico/category.xlsx?dl=1')
df_sales = pd.read_excel('https://www.dropbox.com/s/7359pamv5jcq1cl/sales.xlsx?dl=1')

In [ ]:
def keep_cols(DataFrame, keep_these):
    drop_these = list(set(list(DataFrame)) - set(keep_these))
    return DataFrame.drop(drop_these, axis = 1)

In [ ]:
df_sales = df_sales.pipe(keep_cols, ['UPC', 'BASE_PRICE', 'PRICE', 'UNITS'])
df_category = df_category.pipe(keep_cols, ['UPC', 'CATEGORY'])

In [ ]:
df_sales.head()

In [ ]:
unique_upc = df_sales['UPC'].unique()

In [ ]:
corr_df = df_sales[df_sales['UPC'] == unique_upc[2]]
del corr_df['UPC']
corr_df.corr()

In [ ]:
def get_optimal_price(current_df):
    current_df['demand'] = current_df['BASE_PRICE'] - current_df['PRICE']
    df = current_df[current_df['demand'] > 0]
    df['profit'] = df['demand'] * df['UNITS']
    df['zscore'] = np.abs(stats.zscore(df['PRICE']))
    df = df[df['zscore'] < 3]
    df_std = np.std(df['PRICE']) / 2
    index_max = df['profit'].idxmax()
    df_max = df.loc[index_max, 'PRICE']
    price_arr = df[(df['PRICE'] > df_max - df_std) & (df['PRICE'] < df_max + df_std)]['PRICE']
    return np.mean(price_arr)

In [ ]:
upc_df = pd.DataFrame(columns=['upc', 'price'])
for upc in unique_upc:
    df = df_sales[df_sales['UPC'] == upc]
    upc_df = upc_df.append({'upc': str(upc), 'price': get_optimal_price(df)}, ignore_index=True)

In [ ]:
upc_df

In [ ]:
full_df = pd.merge(df_sales, df_category, on='UPC')

In [ ]:
full_df.head()

In [ ]:
unique_category = full_df['CATEGORY'].unique()

In [ ]:
category_df = pd.DataFrame(columns=['category', 'price'])
for category in unique_category:
    df = full_df[full_df['CATEGORY'] == category]
    del df['CATEGORY']
    category_df = category_df.append({'category': category, 'price': get_optimal_price(df)}, ignore_index=True)

In [ ]:
category_df